In [ ]:
from transformers import AutoTokenizer

base_model = "beomi/KoAlpaca-Polyglot-5.8B"
lora_model_path = "/content/beomi_llm"

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(lora_model_path, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token  # 필수

# 베이스 모델 로드 + LoRA 적용
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map="auto",
    load_in_8bit=True
)
model = PeftModel.from_pretrained(model, lora_model_path)

model.eval()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def generate(prompt, max_new_tokens):
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items() if k != "token_type_ids"}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.5,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
import re

def clean_and_trim_response(prompt, generated_text):
    # prompt 이후 부분만 추출
    response = generated_text[len(prompt):].strip()

    # 불필요한 토큰 제거
    response = re.sub(r'(input:|output:)', '', response).strip()

    # 마침표 기준으로 문장 자르기 후 첫 문장만 사용
    sentences = re.split(r'(?<=[.!?])\s+', response)
    return sentences[0] if sentences else response


input_text = "오늘 밥을 뭘 먹는게 좋을까?"
prompt = f"input: {input_text}\noutput:"
generated = generate(prompt, 100)

result = clean_and_trim_response(prompt, generated)
print(result)

In [ ]:
from gtts import gTTS

# 언어 설정
language = 'ko'

# gTTS 객체 생성
speech = gTTS(text=result, lang=language, slow=False)

# 음성 파일로 저장
speech.save("/content/bap.wav")